In [3]:
import json
import numpy as np
import jieba

Labels = []
with open('./Answers_cn/QA_cn.jsonl', 'r', encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        Labels.append(data['A'])
Answers = Labels[:]

# 1. Rouge

In [11]:
from rouge_chinese import Rouge


def Rouge_Score(hypothesis, reference):
    rouger = Rouge()
    hypothesis_cut = [' '.join(jieba.cut(item)) for item in hypothesis]
    reference_cut = [' '.join(jieba.cut(item)) for item in reference]
    scores = rouger.get_scores(hypothesis_cut, reference_cut)
    arr_1 = np.array([[item['rouge-1']['r'], item['rouge-1']['p'], item['rouge-1']['f']] for item in scores])
    avg_r_1, avg_p_1, avg_f_1 = np.mean(arr_1[:, 0]), np.mean(arr_1[:, 1]), np.mean(arr_1[:, 2])
    arr_2 = np.array([[item['rouge-2']['r'], item['rouge-2']['p'], item['rouge-2']['f']] for item in scores])
    avg_r_2, avg_p_2, avg_f_2 = np.mean(arr_2[:, 0]), np.mean(arr_2[:, 1]), np.mean(arr_2[:, 2])
    arr_l = np.array([[item['rouge-l']['r'], item['rouge-l']['p'], item['rouge-l']['f']] for item in scores])
    avg_r_l, avg_p_l, avg_f_l = np.mean(arr_l[:, 0]), np.mean(arr_l[:, 1]), np.mean(arr_l[:, 2])
    avg_scores = {
        'rouge-1': {'r': avg_r_1, 'p': avg_p_1, 'f': avg_f_1},
        'rouge-2': {'r': avg_r_2, 'p': avg_p_2, 'f': avg_f_2}, 
        'rouge-l': {'r': avg_r_l, 'p': avg_p_l, 'f': avg_f_l}
        }
    return avg_scores, arr_l[:, 2]
rouge_scores, rouge_scores_f1 = Rouge_Score(Answers, Labels)
rouge_scores_f1

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# 2. BLEU

In [10]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu


def Bleu_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[list(jieba.cut(s)) for s in sentences] for sentences in references]
    candidates_tokens = [list(jieba.cut(sentence)) for sentence in candidates]
    bleu_scores = [corpus_bleu(references_tokens, candidates_tokens, weights=(n,)) for n in range(1, 5)]
    bleu_1_scores = [sentence_bleu(r, z, weights=(1,)) for r, z in zip(references_tokens, candidates_tokens)]
    ave_scores = {}
    for n, score in enumerate(bleu_scores, start=1):
       ave_scores[f"BLEU-{n}"] = score
    return ave_scores, bleu_1_scores

Bleu_Scores(Answers, Labels)

({'BLEU-1': 1.0, 'BLEU-2': 1.0, 'BLEU-3': 1.0, 'BLEU-4': 1.0},
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0])

# 3. NIST

In [11]:
from nltk.translate.nist_score import corpus_nist, sentence_nist

def Nist_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[list(jieba.cut(s)) for s in sentences] for sentences in references]
    candidates_tokens = [list(jieba.cut(sentence)) for sentence in candidates]

    nist_5 = corpus_nist(references_tokens, candidates_tokens, n=5)

    nist_5_scores = [sentence_nist(r, z, n=min(len(r), 5)) for r, z in zip(references_tokens, candidates_tokens)]

    return nist_5 , nist_5_scores

Nist_Scores(Answers, Labels)

(10.603531739381001,
 [4.087462841250341,
  4.20184123230257,
  4.523561956057012,
  4.070656113151928,
  4.7606479232901,
  4.83292334597596,
  4.606739354015321,
  4.563856189774724,
  4.863463521783096,
  4.892147223664536,
  5.004886164091842,
  4.885129041304624,
  4.453660689688185,
  4.418295834054491,
  0.0,
  2.8073549220576046,
  2.584962500721156,
  2.0,
  2.8073549220576046,
  2.0,
  2.8073549220576046,
  2.321928094887362,
  2.584962500721156,
  2.321928094887362,
  1.0,
  1.5849625007211563,
  2.0,
  1.0,
  1.0,
  2.321928094887362,
  2.8073549220576046,
  2.0,
  2.8073549220576046,
  3.875,
  5.012479890745559,
  4.941744485170483,
  3.459431618637297,
  4.142664355548848,
  4.087462841250341,
  3.584962500721156,
  4.349648912578751,
  4.152391277629866,
  3.98418371977919,
  4.546593564294941,
  4.4181572881564195,
  4.786425874087821,
  3.4182958340544896,
  4.293660689688185,
  4.161978179679555,
  4.134336113194452,
  3.584962500721156,
  3.0,
  3.7004397181410913,


# 4. Meteor

In [13]:
from nltk.translate.meteor_score import meteor_score

def Meteor_Scores(candidates, references):
    references  = np.array(references).reshape(-1, 1)
    references_tokens = [[list(jieba.cut(s)) for s in sentences] for sentences in references]
    candidates_tokens = [list(jieba.cut(sentence)) for sentence in candidates]

    meteor_scores = [meteor_score(ref, candidate) for ref, candidate in zip(references_tokens, candidates_tokens)]
    return np.mean(meteor_scores).item(), meteor_scores

Meteor_Scores(Answers, Labels)

(0.992257932186208,
 [0.999898229187869,
  0.9999460101500918,
  0.999958905235473,
  0.9999460101500918,
  0.9999832164076399,
  0.9999927453171021,
  0.9999745973682873,
  0.999968,
  0.9999948631544341,
  0.9999892832647462,
  0.9999941303531179,
  0.9999901289163524,
  0.999968,
  0.9999638310185185,
  0.5,
  0.9985422740524781,
  0.9976851851851852,
  0.9921875,
  0.9985422740524781,
  0.9921875,
  0.9985422740524781,
  0.996,
  0.9976851851851852,
  0.996,
  0.9375,
  0.9814814814814815,
  0.9921875,
  0.9375,
  0.9375,
  0.996,
  0.9985422740524781,
  0.9921875,
  0.9985422740524781,
  0.9998779296875,
  0.9999948631544341,
  0.9999945130315501,
  0.9996243425995492,
  0.9999271030762502,
  0.999898229187869,
  0.9997106481481481,
  0.999958905235473,
  0.9999530428249437,
  0.9999375,
  0.9999715521165226,
  0.9999794989544467,
  0.9999883381924198,
  0.9997106481481481,
  0.999968,
  0.9999715521165226,
  0.9999745973682873,
  0.9997106481481481,
  0.9990234375,
  0.9997724169

# 5. Bert Score

In [14]:
from bert_score import score


def Bert_Score(candidates, references):

    P, R, F1 = score(candidates, references, model_type="roberta-large", verbose=False)

    return {"BERT-P:": float(np.mean(P.numpy())), "BERT-R:": float(np.mean(R.numpy())), "BERT-F1:": float(np.mean(F1.numpy()))}, F1.numpy()

Bert_Score(Answers, Labels)


Some weights of the model checkpoint at D:\yuyouyu\CODE\QA_Evaluate\models--roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


({'BERT-P:': 1.0, 'BERT-R:': 1.0, 'BERT-F1:': 1.0},
 array([1.        , 0.9999999 , 1.        , 1.0000001 , 1.        ,
        0.9999999 , 0.9999999 , 1.0000001 , 1.        , 1.0000001 ,
        1.        , 0.9999999 , 1.        , 0.9999999 , 0.99999994,
        1.0000001 , 1.        , 1.0000001 , 1.        , 1.        ,
        0.9999999 , 1.        , 0.9999999 , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.0000001 , 1.        , 1.        , 0.99999994, 1.        ,
        1.        , 1.        , 0.9999999 , 1.        , 1.        ,
        0.9999999 , 1.        , 0.99999994, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.99999994,
        1.0000001 , 0.99999994, 1.        , 0.99999994, 1.        ,
        0.99999994, 0.9999999 , 1.        , 0.9999999 , 1.      

In [26]:
from pycocoevalcap.cider.cider import Cider

def Cider_Score(candidates, references):

    candidates = {i: [' '.join(jieba.cut(sentence))] for i, sentence in enumerate(candidates)}
    references = {i: [' '.join(jieba.cut(sentence))] for i, sentence in enumerate(references)}
    cider_scorer = Cider()
    # 计算CIDEr分数
    ave_score, scores = cider_scorer.compute_score(candidates, references)
    return ave_score.item(), scores


Cider_Score(Answers, Labels)


{0: ['多纳 尔 在 面对 强大 敌人 和 凶狠 神迹 怪物 时 能够 面不改色 地 从容 解决 。'], 1: ['当 多纳 尔 被 同伴 夸奖 时 ， 他会 露出 近似于 害羞 的 神情 ， 避开 对方 赞许 的 眼神 。'], 2: ['多纳 尔 性格 豪爽 、 不拘小节 ， 有时 会 因为 自己 过于 风风火火 的 行事 风格 而 遇到 一些 误解 和 小麻烦 。'], 3: ['多纳 尔 有着 发达 的 肌肉 ， 配合 他 高大 的 个头 和 粗犷 的 外貌 ， 看起来 有些 难以接近 。'], 4: ['多纳 尔 最 喜欢 的 地方 是 健身房 ， 他 有 很多 健身 心得 ， 也 很 喜欢 在 闲暇 时刻 “ 无意间 ” 展示 自己 努力 锻炼 的 成果 。'], 5: ['多纳 尔 不 喜欢 太甜 的 东西 ， 比如 甜酒 。 但 如果 是 浓郁 的 佳酿 ， 他 倒 是 来者不拒 。 在 他 的 家乡 ， 大家 都 说 只有 烈酒 才能 配得 上 最 勇敢 的 战士 。'], 6: ['瑞雯 通过 让 多纳 尔去 后勤部门 做 志愿者 和 去 温室 里 照顾 药草 等 惩罚 ， 磨练 出 了 多纳 尔 的 稳重 和 耐心 。'], 7: ['多纳 尔 没有 为了 证明 自己 而 盲目 地冲 到 最 前面 ， 而是 学会 了 先 观察 情况 ， 考虑 周全 后 行动 。'], 8: ['虽然 多纳 尔 并 不是 十全十美 的 ， 但 对于 他 的 小队 成员 来说 ， 他 是 最好 的 、 最 可靠 的 队长 。 他 遵守规则 并 肩负 更 多 的 责任 ， 收敛 了 冲动 ， 学会 了 在 行动 前 思考 。'], 9: ['多纳 尔 通过 使用 技能 < B702 > 爆雷 天降 < / > 进行 攻击 的 同时 ， 为 自身 附加 免疫 和 防御 提升 状态 ， 进而 强化 自己 的 生存 和 输出 能力 。'], 10: ['多纳 尔 的 被动 技能 < B702 > 雷神 之怒 < / > 会 在 回合 结束 后 ， 让 多纳 尔 进入 引雷 状态 ， 在 自身 新 回合 开始 时 ， 发动 雷霆 之力 ， 对 敌方 全体 造成 大量 伤害 。'], 11: ['技能 < B702 > 爆雷 天降 < / > 在 对 一个 敌人 进行 

(9.75,
 array([10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. ,  2.5, 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. ,  5. ,  7.5, 10. ,  5. ,  5. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
        10. ]))